# Finding the best locality based on your interest 

## Background

I used to live in California. When I had to move to Austin for a job change, it was challenging for me to find a locality of my interest. I am early bird and wanted to start my day with a bit of exercise. I like to go to gym or jog in a park early in the morning. Weekends I wanted to spend time with my friends in a restaurant or a bar. I struggle a lot to find a place. (I finally found a place after lot of search)

Enough about me. As I was trying to find a real-life problem to solve as part of Capstore Project, I just remembered my struggle. I started thinking why I can’t solve this problem using Data Science today. I did a bit to exploration to see what kind of data I need, how to approach, etc.

This will help a lot of people who are migrating from one location to another.

## About the Data

When I was searching for data, I found about www.unitedstateszipcodes.org . They have all the US Zip code information along with location information. This has added advantage for me as I don’t have to go and find the map location. Here is a sample of the data

| zip | type     | decommissioned | primary_city | acceptable_cities | unacceptable_cities                                                                                                                                                                                                                                                                                                                                                                                                                                                                         | state | county              | timezone            | area_codes | world_region | country | latitude | longitude | irs_estimated_population_2015 |
|-----|----------|----------------|--------------|-------------------|---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|-------|---------------------|---------------------|------------|--------------|---------|----------|-----------|-------------------------------|
| 501 | UNIQUE   | 0              | Holtsville   |                   | I R S Service Center                                                                                                                                                                                                                                                                                                                                                                                                                                                                        | NY    | Suffolk County      | America/New_York    | 631        | NA           | US      | 40.81    | -73.04    | 562                           |
| 544 | UNIQUE   | 0              | Holtsville   |                   | Irs Service Center                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | NY    | Suffolk County      | America/New_York    | 631        | NA           | US      | 40.81    | -73.04    | 0                             |
| 601 | STANDARD | 0              | Adjuntas     |                   | Colinas Del Gigante, Jard De Adjuntas, Urb San Joaquin                                                                                                                                                                                                                                                                                                                                                                                                                                      | PR    | Adjuntas Municipio  | America/Puerto_Rico | 787,939    | NA           | US      | 18.16    | -66.72    | 0                             |
| 602 | STANDARD | 0              | Aguada       |                   | Alts De Aguada, Bo Guaniquilla, Comunidad Las Flores, Ext Los Robles, Urb Isabel La Catolica                                                                                                                                                                                                                                                                                                                                                                                                | PR    | Aguada Municipio    | America/Puerto_Rico | 787,939    | NA           | US      | 18.38    | -67.18    | 0                             |
| 603 | STANDARD | 0              | Aguadilla    | Ramey             | Bda Caban, Bda Esteves, Bo Borinquen, Bo Ceiba Baja, Ext El Prado, Ext Marbella, Repto Jimenez, Repto Juan Aguiar, Repto Lopez, Repto Tres Palmas, Sect Las Villas, Urb Borinquen, Urb El Prado, Urb Esteves, Urb Garcia, Urb Las Americas, Urb Las Casitas Country Club, Urb Maleza Gdns, Urb Marbella, Urb Ramey, Urb Rubianes, Urb San Carlos, Urb Santa Marta, Urb Victoria, Villa Alegria, Villa Linda, Villa Lydia, Villa Universitaria, Villas De Almeria, Vista Alegre, Vista Verde | PR    | Aguadilla Municipio | America/Puerto_Rico | 787        | NA           | US      | 18.43    | -67.15    | 0                             |
| 604 | PO BOX   | 0              | Aguadilla    | Ramey             |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | PR    |                     | America/Puerto_Rico |            | NA           | US      | 18.43    | -67.15    | 0                             |
| 605 | PO BOX   | 0              | Aguadilla    |                   |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | PR    |                     | America/Puerto_Rico |            | NA           | US      | 18.43    | -67.15    | 0                             |
| 606 | STANDARD | 0              | Maricao      |                   | Urb San Juan Bautista                                                                                                                                                                                                                                                                                                                                                                                                                                                                       | PR    | Maricao Municipio   | America/Puerto_Rico | 787,939    | NA           | US      | 18.18    | -66.98    | 0                             |
| 610 | STANDARD | 0              | Anasco       |                   | Brisas De Anasco, Est De Valle Verde, Jard De Anasco, Paseo Del Valle, Repto Daguey, Urb San Antonio                                                                                                                                                                                                                                                                                                                                                                                        | PR    | Anasco Municipio    | America/Puerto_Rico | 787        | NA           | US      | 18.28    | -67.14    | 0                             |
| 611 | PO BOX   | 0              | Angeles      |                   |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | PR    |                     | America/Puerto_Rico |            | NA           | US      | 18.28    | -66.79    | 0                             |

I can find the near by places from Foursquare for each of these zip code for the specific city/location. And try to see which zip code has all or most of my interest with-in specified distance. 